In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
# import s3fs
import re
from pelicanfs.core import PelicanFileSystem, PelicanMap,OSDFFileSystem 
import fsspec.implementations.http as fshttp
import geopandas as gpd
from pystac_client import Client
from odc.stac import stac_load
from rasterio.mask import mask
import rasterio
from shapely.geometry import box

In [2]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client as dask_client
from dask.distributed import performance_report

### Use the GeoJSON file to get the Area of Interest and query the STAC catalog for items

In [3]:
# Load the GeoJSON file
geojson_path = '/glade/u/home/harshah/osdf_examples/3100180240.geojson' 
gdf = gpd.read_file(geojson_path)

# Display the loaded GeoDataFrame
print(gdf)

                     id  SUB_AREA  COAST     PFAF_ID  DIST_MAIN    HYBAS_ID  \
0  00140000000000002983     128.1      0  3512704524      784.3  3100180240   

   DIST_SINK   NEXT_DOWN  ORDER  ENDO    MAIN_BAS   NEXT_SINK   SORT  UP_AREA  \
0      784.3  3100180230      4     0  3100009670  3100009670  66318    128.1   

                                            geometry  
0  POLYGON ((134.51667 66.87917, 134.51752 66.875...  


In [4]:
# Extract AOI geometry
aoi_geometry = gdf.geometry.iloc[0]
aoi_bounds   = aoi_geometry.bounds  # (minx, miny, maxx, maxy)

# Get AOI centroid for visualization
centroid  = aoi_geometry.centroid
long, lat = centroid.x, centroid.y

# Print the bounding box to verify
print("Bounding Box:", aoi_bounds)

Bounding Box: (134.51666686838126, 66.7708333115583, 134.9046488751149, 66.97083294070578)


In [5]:
# Connect to the Earth Search STAC API (Sentinel-2 Level-2A COGs are available here)
catalog_url = "https://earth-search.aws.element84.com/v1"
catalog     = Client.open(catalog_url)

# Define the date range as strings
start_date      = "2019-01"
end_date        = "2023-02"

# Define cloud cover threshold
cloud_cover_max = 0.05  # 5% cloud cover threshold
#cloud_cover_max = 0.20  # 20% cloud cover threshold

# Perform the search
search = catalog.search(
                 collections=["sentinel-2-l2a"],
                 bbox=aoi_bounds,
                 datetime=f"{start_date}/{end_date}",
                 #datetime="2022-06-01/2022-09-30",
                 query={"eo:cloud_cover": {"lt": cloud_cover_max * 100}}
                )

# Get all matching items
items = list(search.items())
print(f"Found {len(items)} matching items.")

# for item in items[:1]:  # Print details for the first few items
#     print(item.to_dict())

# Process each dataset with a simple count message
# for idx, item in enumerate(items, start=1,stop=2):
#     print(f"Processing dataset #{idx}")

Found 257 matching items.


In [6]:
from shapely.geometry import mapping

# Reproject AOI to match raster CRS
from pyproj import CRS
from geopandas import GeoSeries

### Apply various masks and calculate NDVI

In [7]:
def clp(image_src, aoi_geometry):
    """
    Clip a raster image to the Area of Interest (AOI).

    Parameters:
    - image_src: Open rasterio dataset.
    - aoi_geometry: AOI geometry as a GeoJSON-like object.

    Returns:
    - Clipped image array and updated metadata.
    """
    out_image, out_transform = mask(image_src, [aoi_geometry], crop=True)
    out_meta = image_src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    return out_image, out_meta

def maskWater(image, water_mask):
    """
    Masks out water pixels using the MODIS water mask.

    Parameters:
    - image: Raster image to mask.
    - water_mask: Water mask raster.

    Returns:
    - Water-masked image.
    """
    water = water_mask.read(1)  # Read the water mask
    mask = water < 1  # Mask water pixels (water < 1)
    image_masked = np.where(mask, image, np.nan)
    return image_masked

def maskS2snow(image, snow_prob):
    """
    Masks snow pixels using the MSK_SNWPRB (Snow Probability Mask).

    Parameters:
    - image: Raster image to mask.
    - snow_prob: Snow probability raster.

    Returns:
    - Snow-masked image.
    """
    snow = snow_prob.read(1)  # Read the snow probability mask
    mask = snow < 0.009  # Mask snow pixels (snow probability < 0.9%)
    image_masked = np.where(mask, image, np.nan)
    return image_masked

def maskWhite(image, b2, b3, b4):
    """
    Masks white pixels by converting RGB to grayscale and applying a threshold.

    Parameters:
    - image: Raster image to mask.
    - b2, b3, b4: Blue, Green, and Red bands respectively.

    Returns:
    - Grayscale-masked image.
    """
    # Convert RGB to grayscale
    grayscale = (0.3 * b4.read(1) + 0.59 * b3.read(1) + 0.11 * b2.read(1)) * 1e4
    mask = grayscale <= 2000  # Mask white pixels (grayscale > 2000)
    image_masked = np.where(mask, image, np.nan)
    return image_masked

In [8]:
# Loop through each item in the STAC query
for idx, item in enumerate(items, start=1):
    print(f"Processing dataset #{idx}")

    # Check required assets
    required_assets = ["red", "green", "blue", "scl"]
    if not all(asset in item.assets for asset in required_assets):
        print(f"Skipping dataset #{idx}: Missing required assets.")
        continue

    # Get asset URLs
    red_url = item.assets["red"].href
    green_url = item.assets["green"].href
    blue_url = item.assets["blue"].href
    scl_url = item.assets["scl"].href

    # Reproject AOI to match raster CRS
    with rasterio.open(red_url) as src:
        raster_crs = CRS(src.crs)
    aoi_geometry_reprojected = GeoSeries(aoi_geometry).set_crs(gdf.crs).to_crs(raster_crs)
    aoi_geometry_reprojected = mapping(aoi_geometry_reprojected.iloc[0])

    # Open and clip bands
    with rasterio.open(red_url) as red_src, \
         rasterio.open(green_url) as green_src, \
         rasterio.open(blue_url) as blue_src, \
         rasterio.open(scl_url) as scl_src:

        raster_bounds = box(*red_src.bounds)
        if not aoi_geometry.intersects(raster_bounds):
            print(f"Warning: AOI does not intersect the bounds of {red_url}. Skipping.")
            continue

        # Clip all RGB and SCL bands to AOI
        red_clipped, red_meta = clp(red_src, aoi_geometry)
        green_clipped, _      = clp(green_src, aoi_geometry)
        blue_clipped, _       = clp(blue_src, aoi_geometry)
        scl_clipped, _        = clp(scl_src, aoi_geometry)

        # Clip water_mask if available
        if water_mask_available:
            with rasterio.open(water_mask_url) as water_mask_src:
                water_mask_clipped, _ = clp(water_mask_src, aoi_geometry)

        # Apply maskS2clouds
        red_masked = maskS2clouds(red_clipped, scl_clipped)
        green_masked = maskS2clouds(green_clipped, scl_clipped)
        blue_masked = maskS2clouds(blue_clipped, scl_clipped)

        # Apply maskWater if available
        if water_mask_available:
            red_masked = maskWater(red_masked, water_mask_clipped)
            green_masked = maskWater(green_masked, water_mask_clipped)
            blue_masked = maskWater(blue_masked, water_mask_clipped)

        # Apply maskWhite
        red_final = maskWhite(red_masked, blue_masked, green_masked, red_masked)
        green_final = maskWhite(green_masked, blue_masked, green_masked, red_masked)
        blue_final = maskWhite(blue_masked, blue_masked, green_masked, red_masked)

        print(f"Dataset #{idx} processed. Ready for visualization or further analysis.")

Processing dataset #1
Processing dataset #2
Processing dataset #3
Processing dataset #4
Processing dataset #5
Processing dataset #6
Processing dataset #7
Processing dataset #8
Processing dataset #9
Processing dataset #10
Processing dataset #11
Processing dataset #12
Processing dataset #13
Processing dataset #14
Processing dataset #15
Processing dataset #16
Processing dataset #17
Processing dataset #18
Processing dataset #19
Processing dataset #20
Processing dataset #21
Processing dataset #22
Processing dataset #23
Processing dataset #24



KeyboardInterrupt



In [ ]:
def visualize_rgb(red_band, green_band, blue_band, title="RGB Composite"):
    """
    Visualizes an RGB composite of raster bands using matplotlib.

    Parameters:
    - red_band, green_band, blue_band (numpy array): Red, Green, and Blue bands.
    - title (str): Title for the plot.
    """
    rgb = np.dstack((
        np.clip(red_band, 0, 1),  # Normalize reflectance between 0-1
        np.clip(green_band, 0, 1),
        np.clip(blue_band, 0, 1)
    ))
    plt.figure(figsize=(10, 8))
    plt.imshow(rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()

# Example: Visualize RGB composite
visualize_rgb(red_final, green_clipped, blue_clipped, title="Processed RGB Composite")
